# Description

It combines all clustering solutions generated into a single consolidated solution using consensus clustering.

# Environment variables

In [1]:
from IPython.display import display

# set numpy n_jobs to 1, since I'll be using n_jobs later
NUMPY_N_JOBS = 1
display(NUMPY_N_JOBS)

1

In [2]:
%env MKL_NUM_THREADS=$NUMPY_N_JOBS
%env OPEN_BLAS_NUM_THREADS=$NUMPY_N_JOBS
%env NUMEXPR_NUM_THREADS=$NUMPY_N_JOBS
%env OMP_NUM_THREADS=$NUMPY_N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [5]:
np.random.seed(0)

In [6]:
RANDOM_STATES_ITER = iter(np.random.randint(0, np.iinfo(np.int32).max, size=100000))

## Consensus clustering

In [7]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 60

display(CLUSTERING_OPTIONS)

{'K_MIN': 2, 'K_MAX': 60}

In [8]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

# Load ensemble

In [9]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble.npy')

In [10]:
full_ensemble = np.load(output_file)

In [11]:
display(full_ensemble.shape)

(4428, 3749)

# Load ensemble coassociation distance matrix

In [12]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble_coassoc_matrix.npy')

In [13]:
ensemble_coassoc_matrix = np.load(output_file)

In [14]:
display(ensemble_coassoc_matrix.shape)

(3749, 3749)

In [15]:
display(ensemble_coassoc_matrix)

array([[0.        , 0.05638587, 0.05412138, ..., 0.59986413, 0.66148571,
        0.66757246],
       [0.05638587, 0.        , 0.02740036, ..., 0.60144928, 0.66011429,
        0.66530797],
       [0.05412138, 0.02740036, 0.        , ..., 0.59805254, 0.66125714,
        0.66281703],
       ...,
       [0.59986413, 0.60144928, 0.59805254, ..., 0.        , 0.6699339 ,
        0.66531165],
       [0.66148571, 0.66011429, 0.66125714, ..., 0.6699339 , 0.        ,
        0.58627764],
       [0.66757246, 0.66530797, 0.66281703, ..., 0.66531165, 0.58627764,
        0.        ]])

# Consensus clustering

In [16]:
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm

from clustering.ensembles.utils import (
    run_method_and_compute_agreement,
)
from clustering.ensembles.eac import (
    eac_single_coassoc_matrix,
    eac_complete_coassoc_matrix,
    eac_average_coassoc_matrix,
)
from clustering.ensembles.spectral import scc

Define spectral consensus clustering methods with delta values found in pre-analysis:

In [17]:
def scc_020(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.20,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_025(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.25,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_030(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.30,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_050(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.50,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )

In [18]:
all_consensus_methods = set(
    (
        eac_single_coassoc_matrix,
        eac_complete_coassoc_matrix,
        eac_average_coassoc_matrix,
        scc_020,
        scc_025,
        scc_030,
        scc_050,
    )
)
display(all_consensus_methods)

{<function clustering.ensembles.eac.eac_average_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_complete_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_single_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function __main__.scc_020(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_025(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_030(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_050(coassoc_distance_matrix, k, **kwargs)>}

In [19]:
consensus_results = []

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(
            run_method_and_compute_agreement,
            m,
            ensemble_coassoc_matrix,
            full_ensemble,
            k,
            random_state=next(RANDOM_STATES_ITER),
        ): (m.__name__, k)
        for m in all_consensus_methods
        for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1)
    }

    for future in tqdm(as_completed(tasks), total=len(tasks), disable=False, ncols=100):
        method_name, k = tasks[future]
        part, performance_values = future.result()

        method_results = {
            "method": method_name,
            "partition": part,
            "k": k,
        }
        method_results.update(performance_values)

        consensus_results.append(method_results)

  0%|                                                                       | 0/413 [00:00<?, ?it/s]

  0%|▏                                                            | 1/413 [00:22<2:31:26, 22.05s/it]

  0%|▎                                                            | 2/413 [00:23<1:47:57, 15.76s/it]

  1%|▍                                                            | 3/413 [00:45<2:01:11, 17.74s/it]

  1%|▌                                                            | 4/413 [00:47<1:28:24, 12.97s/it]

...

 99%|██████████████████████████████████████████████████████████▌| 410/413 [2:22:43<01:33, 31.17s/it]

100%|██████████████████████████████████████████████████████████▋| 411/413 [2:23:03<00:55, 27.74s/it]

100%|██████████████████████████████████████████████████████████▊| 412/413 [2:23:45<00:31, 31.95s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [2:24:04<00:00, 28.12s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [2:24:04<00:00, 20.93s/it]

In [20]:
consensus_results = pd.DataFrame(consensus_results)

In [21]:
display(consensus_results.shape)

(413, 12)

In [22]:
consensus_results.head()

,method,partition,k,ari_mean,ari_median,ari_std,ami_mean,ami_median,ami_std,nmi_mean,nmi_median,nmi_std
0,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0.028316,0.001172,0.071466,0.037372,0.013340,0.070718,0.039099,0.015125,0.070828
1,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,0.044564,0.001975,0.110040,0.059596,0.022292,0.108635,0.062536,0.025583,0.108613
2,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,0.242846,0.015785,0.359750,0.269175,0.082025,0.326123,0.273511,0.086402,0.324377
3,eac_single_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5,0.246215,0.017924,0.353398,0.274577,0.094328,0.314086,0.279641,0.099375,0.311975
4,eac_single_coassoc_matrix,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",6,0.254425,0.024476,0.344744,0.288252,0.110970,0.297056,0.294332,0.117303,0.294392


## Testing

In [23]:
assert not consensus_results.isna().any().any()

In [24]:
# check that the number of clusters in the partitions are the expected ones
_real_k_values = consensus_results["partition"].apply(lambda x: np.unique(x).shape[0])
display(_real_k_values)
assert np.all(consensus_results["k"].values == _real_k_values.values)

0       2
1       3
2       4
3       5
4       6
       ..
408    56
409    57
410    58
411    59
412    60
Name: partition, Length: 413, dtype: int64

## Save

In [25]:
output_file = Path(RESULTS_DIR, "consensus_clustering_runs.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/consensus_clustering_runs.pkl')

In [26]:
consensus_results.to_pickle(output_file)